In [ ]:
#----import libraries----------------------------------------#
import gym
import numpy as np
from gym import wrappers
import random as rand
import seaborn as sns
import matplotlib.pyplot as plt
#-----set up environment------------------------------------#
env = gym.make('CartPole-v0')

# Random weight generation method

In [ ]:
best_moves = 0
episode_length = []
best_weights = np.zeros(4) #emmpty 4 digit array

for i in range (100):
    new_weights = [rand.uniform(0,1),rand.uniform(0,1),rand.uniform(0,1),rand.uniform(0,1)]
    moves_length = []
    for j in range(100):
        observation = env.reset()
        done = False
        count = 0
        while not done:
            count+=1
            action = 1 if np.dot(observation,new_weights)>0 else  0
            observation,reward,done,_ = env.step(action)
            if done:
                break
        moves_length.append(count)
    avg_moves = float(sum(moves_length)/len(moves_length))
    if avg_moves > best_moves:
        best_moves = avg_moves
        best_weights = new_weights
    episode_length.append(avg_moves)
    if i % 20 == 0:
        print("best moves is ", best_moves)
done = False
counr = 0
env = wrappers.Monitor(env, 'movefiles',force=True) #save files

# Q Learning method

In [ ]:
#converting a continuouss state into a discrete state
MAX_STATES = 10**2
GAMMA = 0.7   #discount factor
ALPHA = 0.001 #Learning rate

#q table disctionary lookup(like weights) gets state with highest reward?
#q table disctionary lookup(like weights) gets state with highest reward?
def q_states_dict(q):
    a,b = max(q.items(), key=lambda k: k[1])
    #print("z",a,b)
    return a,b
# our inputs
def create_bins():
    bins = np.zeros((4,15))
    bins[0] = np.linspace(-2.4,2.4,15)
    bins[1] = np.linspace(-10,10,15)
    bins[2] = np.linspace(-41.8,41.8,15)
    bins[3] = np.linspace(-10,10,15)
    return bins
#digitize bins since we are converting a continuous state to a discrete state

In [ ]:
def assign_bins(observation,bins):
    env_state = np.zeros(4) #creates a single array to store the 4 states
    for i in range(4):
        env_state[i] = np.digitize(observation[i],bins[i])
    return env_state

def get_state_as_string(env_state):
    env_string = ''.join(str(int(e)) for e in env_state)
    return env_string

def get_all_states_string():
    env_states = []
    for i in range(MAX_STATES):
        env_states.append(str(i).zfill(8))
    return env_states

def initialise_q():
    Q = {}
    all_states = get_all_states_string()
    for state in all_states:
        Q[state] = {}
        for action in range(env.action_space.n):
            Q[state][action] = 0 #set rewards
    return Q

In [ ]:
def play_a_game(bins,Q,eps):
    observation = env.reset()
    done = False
    count = 0
    new_states_created = 0
    total_reward = 0
    state = get_state_as_string(assign_bins(observation,bins))
    y = Q.get(state)
    while not done:
        #print("state",state)
        count+=1
        init_random = np.random.uniform(0,1)
        if init_random < eps:
            act = env.action_space.sample() #pick a random action
        else:
            act,_ = q_states_dict(Q[state])
        observation,reward,done,_ = env.step(act)
        total_reward+=reward
        if done and count < 200:
            reward = -1000
        new_state = get_state_as_string(assign_bins(observation,bins))
        a1,max_q_s1a1 = q_states_dict(Q[new_state])
        Q[state][act]= ((1 - ALPHA)*(Q[state][act])) + ALPHA*(reward + GAMMA * max_q_s1a1)
        state,act = new_state, a1
        if done:
            q_t = Q
    return total_reward, count,new_states_created,q_t

def play_multiple(bins, N):
    Q = initialise_q()
    count=[]
    new_state_array = 0
    reward = []
    for n in range(N):
        if n > 1:
            epsilon = 1/np.sqrt(n+1)
        else:
            epsilon = 0
        epsilon = 1/np.sqrt(n+1)
        ep_reward,ep_length, states_created,table = play_a_game(bins,Q,epsilon)
        new_state_array+=states_created
        if n %10000 == 0:
            print(n,epsilon,ep_reward,new_state_array)
        count.append(ep_length)
        reward.append(ep_reward)
    return count,reward,table

In [ ]:
bins = create_bins()
x,y,table = play_multiple(bins,100000)

In [ ]:
table
import pandas as pd
q_tdf =  pd.DataFrame(table)
q_tdf.to_csv("Q_table.csv")

In [ ]:
table

In [ ]:
sum(y)

In [ ]:

y_array = []
for i in range(len(y)):
    y_array.append(i)
y1 = y
x1 = y_array
plt.plot(x1,y,"b") #actual
plt.show()

In [ ]:
N = len(y)
r_avg = np.empty(N)
for t in range(N):
    r_avg[t] = np.mean(y[max(0,t-100):(t+1)])
plt.plot(r_avg)
plt.show()

In [ ]:
for episode in range (20):
    observation = env.reset()
    done = False
    print("-----------------------")
    #time.sleep(1)
    count = 0
    state = get_state_as_string(assign_bins(observation,bins))
    while not done:
        env.render()
        act,_ = q_states_dict(table[state])
        observation,reward,done,_ = env.step(act)
        state = get_state_as_string(assign_bins(observation,bins))
        count+=1
        if done:
            print("counter", count)
            break
            
        state = get_state_as_string(assign_bins(observation,bins))

env.close()
env.close()